# 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# 데이터 불러오기

In [ ]:
data_path='/content/drive/Shareddrives/Multicampus/mimic-iii-clinical-database-1.4/'
# data_path='mimic-iii-clinical-database-1.4/'

In [ ]:
admissions=pd.read_csv(data_path+'ADMISSIONS.csv.gz')
patients=pd.read_csv(data_path+'PATIENTS.csv.gz')
labevents=pd.read_csv(data_path+'LABEVENTS.csv.gz')
d_labitems=pd.read_csv(data_path+'D_LABITEMS.csv.gz')

# admission 중복제거 테이블 생성

In [ ]:
admissions_nodup=admissions.drop_duplicates(['SUBJECT_ID'], keep=False)
adm_pat=admissions_nodup.merge(patients, on='SUBJECT_ID',how='inner')
adm_pat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38983 entries, 0 to 38982
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ROW_ID_x              38983 non-null  int64 
 1   SUBJECT_ID            38983 non-null  int64 
 2   HADM_ID               38983 non-null  int64 
 3   ADMITTIME             38983 non-null  object
 4   DISCHTIME             38983 non-null  object
 5   DEATHTIME             4351 non-null   object
 6   ADMISSION_TYPE        38983 non-null  object
 7   ADMISSION_LOCATION    38983 non-null  object
 8   DISCHARGE_LOCATION    38983 non-null  object
 9   INSURANCE             38983 non-null  object
 10  LANGUAGE              19769 non-null  object
 11  RELIGION              38562 non-null  object
 12  MARITAL_STATUS        29541 non-null  object
 13  ETHNICITY             38983 non-null  object
 14  EDREGTIME             17912 non-null  object
 15  EDOUTTIME             17912 non-null

# labevents 작업

In [ ]:
# adm_pat와 labevents HADM_ID 기준으로 합치기
adm_pat_lab=pd.merge(adm_pat[['HADM_ID']],labevents, how='inner', on='HADM_ID')
adm_pat_lab

HADM_ID    ROW_ID  SUBJECT_ID  ...  VALUENUM VALUEUOM      FLAG
0          165315     10575          22  ...       NaN      NaN       NaN
1          165315     10576          22  ...       NaN      NaN       NaN
2          165315     10577          22  ...       NaN      NaN       NaN
3          165315     10578          22  ...       NaN      NaN       NaN
4          165315     10579          22  ...       NaN      NaN       NaN
...           ...       ...         ...  ...       ...      ...       ...
13469464   122631  27732612       98805  ...     185.0     K/uL       NaN
13469465   122631  27732613       98805  ...      14.5        %       NaN
13469466   122631  27732614       98805  ...       4.2     m/uL  abnormal
13469467   122631  27732615       98805  ...      11.0     K/uL       NaN
13469468   122631  27732616       98805  ...       6.6    ug/mL  abnormal

[13469469 rows x 9 columns]

In [ ]:
# ITEMID unique 개수
len(adm_pat_lab.ITEMID.unique())

696

In [ ]:
# 단위가 없는 것 확인
adm_pat_lab.VALUEUOM.isnull().sum() # 1914688
# (adm_pat_lab.VALUEUOM.isnull().sum()/adm_pat_lab.shape[0])*100 ==> 약11.8%
# --> itemid로 해당 labitem 찾아서 단위 추가하면 될 듯

# 만약 value 자체가 없으면 날리기? (아래 셀)

# value 없는 행 개수
adm_pat_lab.VALUE.isnull().sum() # 576

576

In [ ]:
# value nan 행 확인
adm_pat_lab[['VALUE','VALUENUM','VALUEUOM']][adm_pat_lab.VALUE.isnull()]
adm_pat_lab[adm_pat_lab.VALUE.isnull()]

HADM_ID    ROW_ID  SUBJECT_ID  ITEMID  ... VALUE VALUENUM  VALUEUOM FLAG
7654       123421    238928         370   50911  ...   NaN      NaN     ng/mL  NaN
7688       123421    238963         370   50911  ...   NaN      NaN     ng/mL  NaN
69676      139824    241247         377   50911  ...   NaN      NaN     ng/mL  NaN
81331      173910    268124         408   50911  ...   NaN      NaN     ng/mL  NaN
81493      173910    266546         408   50911  ...   NaN      NaN     ng/mL  NaN
...           ...       ...         ...     ...  ...   ...      ...       ...  ...
13271869   175706  27059335       93640   50821  ...   NaN      NaN     mm Hg  NaN
13289670   102972  26724971       91086   51196  ...   NaN      NaN       NaN  NaN
13323181   163517  26652466       90552   51196  ...   NaN      NaN       NaN  NaN
13325389   146682  26656151       90609   51196  ...   NaN      NaN       NaN  NaN
13402451   118770  27147141       94300   51196  ...   NaN      NaN       NaN  NaN

[576 rows x 9 columns]

In [ ]:
# value = nan인 행 삭제하는 코드 
# adm_pat_lab=adm_pat_lab.drop(adm_pat_lab[adm_pat_lab.VALUE.isnull()].index, axis=0)

In [ ]:
adm_pat_lab_item=adm_pat_lab.merge(d_labitems,on=["ITEMID"])

In [ ]:
adm_pat_lab_item.drop(["ROW_ID_x","ROW_ID_y"],axis=1,inplace=True)
adm_pat_lab_item

HADM_ID  SUBJECT_ID  ITEMID  ...  FLUID    CATEGORY  LOINC_CODE
0          165315          22   51071  ...  Urine   Chemistry      3349-8
1          144383         372   51071  ...  Urine   Chemistry      3349-8
2          190797          52   51071  ...  Urine   Chemistry      3349-8
3          172056          64   51071  ...  Urine   Chemistry      3349-8
4          143430          65   51071  ...  Urine   Chemistry      3349-8
...           ...         ...     ...  ...    ...         ...         ...
13469464   175201       98573   51169  ...  Blood  Hematology         NaN
13469465   175201       98573   51170  ...  Blood  Hematology         NaN
13469466   175201       98573   51170  ...  Blood  Hematology         NaN
13469467   175201       98573   51170  ...  Blood  Hematology         NaN
13469468   175201       98573   51170  ...  Blood  Hematology         NaN

[13469469 rows x 12 columns]

In [ ]:
# unit이 1개 이상인 ITEMID를 unitdup 리스트에 저장
itemids=adm_pat_lab_item.ITEMID.unique()
unitdup=[]

for i in itemids:
    if len(adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()) != 1:
        unitdup.append(i)

In [ ]:
# unit이 1개 이상인 ITEMID 개수
len(unitdup)

118

In [ ]:
for i in unitdup:
    a = adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()
    print(i, a) 

51464 ['EU/dL' 'mg/dL' nan]
51466 ['mg/dL' nan]
51476 ['#/hpf' nan]
51478 ['mg/dL' nan]
51484 ['mg/dL' nan]
51491 ['units' nan]
51492 ['mg/dL' nan]
51493 ['#/hpf' nan]
51498 [' ' nan]
51514 ['EU/dL' 'mg/dL' nan]
51516 ['#/hpf' nan]
50861 ['IU/L' nan]
50863 ['IU/L' nan]
50868 ['mEq/L' nan]
50878 ['IU/L' nan]
50882 ['mEq/L' nan]
50885 ['mg/dL' nan]
50893 ['mg/dL' nan]
50902 ['mEq/L' nan]
50912 ['mg/dL' nan]
50931 ['mg/dL' nan]
50960 ['mg/dL' nan]
50970 ['mg/dL' nan]
50971 ['mEq/L' nan]
50983 ['mEq/L' nan]
51006 ['mg/dL' nan]
51146 ['%' nan]
51200 ['%' nan]
51221 ['%' nan]
51222 ['g/dL' nan]
51244 ['%' nan]
51248 ['pg' nan]
51249 ['%' nan]
51250 ['fL' nan]
51254 ['%' nan]
51256 ['%' nan]
51265 ['K/uL' nan]
51274 ['SECONDS' 'sec' nan]
51275 ['sec' nan]
51277 ['%' nan]
51279 ['m/uL' nan]
51301 ['K/uL' nan]
50802 ['mEq/L' nan]
50804 ['mEq/L' 'MEQ/L' nan]
50818 ['mm Hg' 'MM HG' nan]
50820 ['units' 'UNITS' nan]
50821 ['mm Hg' 'MM HG' nan]
50993 ['uU/ML' 'uIU/mL' nan]
50810 ['%' nan]
50811 ['g/

In [ ]:
len_2=[]
for i in unitdup:
    a = adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()
    if len(a) ==2:
        len_2.append(i)

In [ ]:
len(len_2)

106

In [ ]:
# 기존의 unit 열 복사해두기
adm_pat_lab_item['raw_VALUEUOM']=adm_pat_lab_item['VALUEUOM']
adm_pat_lab_item['raw_VALUENUM']=adm_pat_lab_item['VALUENUM']

## len_2에 있는 ITEMID 값 중에 nan 을 단위로 대체

In [ ]:
for x in len_2:
    target=adm_pat_lab_item[adm_pat_lab_item.ITEMID==x]
    idx=target[target.VALUEUOM.isnull()].index
    a = adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==x].unique()
    adm_pat_lab_item.iloc[idx, 6]=a[0]

In [ ]:
# 위 코드 실행 결과 확인
adm_pat_lab_item[['VALUEUOM','raw_VALUEUOM']][adm_pat_lab_item.ITEMID==len_2[0]]
adm_pat_lab_item['raw_VALUEUOM'][adm_pat_lab_item.ITEMID==len_2[0]].unique()
adm_pat_lab_item['VALUEUOM'][adm_pat_lab_item.ITEMID==len_2[0]].unique()

array(['mg/dL'], dtype=object)

In [ ]:
# unit 중복 컬럼 재추출
itemids=adm_pat_lab_item.ITEMID.unique()
unitdup=[]

for i in itemids:
    if len(adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()) != 1:
        unitdup.append(i)

len_2=[]
for i in unitdup:
    a = adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()
    if len(a) ==2:
        len_2.append(i)
len(len_2)

22

## 나머지 unit 통일

In [ ]:
for i in len_2:
    a = adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()
    print(i, a) 

51003 ['ng/mL' 'ng/ml']
50816 [nan '%']
50924 ['ng/ml' 'ng/mL']
50935 ['mg/dL' 'MG/DL']
50980 ['IU/mL' 'I.U.']
51127 ['#/uL' '#/CU MM']
51128 ['#/uL' '#/CU MM']
50864 ['ng/mL' 'nG/ML']
50994 ['uG/DL' 'ug/dL']
50926 ['mIU/mL' 'mIU/L']
50958 ['mIU/mL' 'mIU/L']
51105 ['MG/DL' 'mg/dL']
50990 ['ug/ml' 'ug/mL']
51001 ['ng/dL' 'NG/DL']
50974 ['ng/mL' 'ug/L']
51067 ['mg/24hr' 'mg/24hours']
50896 ['ug/dL' 'uG/DL']
51005 ['Ratio' 'RATIO']
50973 ['ng/mL' 'nG/mL']
50989 ['pg/mL' 'ng/dL']
50916 ['nG/mL' 'ug/dL']
50833 [nan 'mEq/L']


In [ ]:
def change_unit(itemid, unit):
    idx=adm_pat_lab_item[adm_pat_lab_item.ITEMID==itemid].index
    a = adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==itemid].unique()
    adm_pat_lab_item.iloc[idx, 6]=unit

In [ ]:
change_unit(51003 , 'ng/mL')
change_unit(50816, '%')
change_unit(50924, 'ng/ml')
change_unit(50935, 'mg/dL')
change_unit(50980, 'IU/mL')
change_unit(51127 , '#/uL')
change_unit(51128 , '#/uL')
change_unit(50864 , 'ng/mL')
change_unit(50994 , 'ug/dL')
change_unit(50926 , 'mIU/mL')
change_unit(50958 , 'mIU/mL')
change_unit(51105 , 'mg/dL')
change_unit(50990 , 'ug/mL')
change_unit(51001 , 'ng/dL')
change_unit(50974 , 'ng/mL')
change_unit(51067 , 'mg/24hr')
change_unit(50896 , 'ug/dL')
change_unit(51005 , 'Ratio')
change_unit(50973 , 'ng/mL')
change_unit(50989 , 'pg/mL')
change_unit(50833 , 'mEq/L')
change_unit(50916 , 'ug/dL')

# specific gravity: 비율 값
change_unit(51498 , 'Ratio')

In [ ]:
# unit 중복 컬럼 재추출 ==> 확인작업
itemids=adm_pat_lab_item.ITEMID.unique()
unitdup=[]

for i in itemids:
    if len(adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()) != 1:
        unitdup.append(i)

len_2_f=[]
for i in unitdup:
    a = adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()
    if len(a) ==2:
        len_2_f.append(i)
len(len_2_f)

0

In [ ]:
len_2_f

[]

In [ ]:
adm_pat_lab_item[['VALUE','VALUENUM','VALUEUOM','LABEL']][adm_pat_lab_item.ITEMID==50916]
adm_pat_lab_item[adm_pat_lab_item.ITEMID==50916]

HADM_ID  SUBJECT_ID  ITEMID  ... LOINC_CODE raw_VALUEUOM  raw_VALUENUM
13466079   115681        5933   50916  ...     2191-5        nG/mL         118.0
13466080   102305        8469   50916  ...     2191-5        ug/dL          71.0
13466081   183240       18807   50916  ...     2191-5        ug/dL          22.0
13466082   184811       27372   50916  ...     2191-5        ug/dL          14.0
13466083   181675       26542   50916  ...     2191-5        ug/dL          15.0
13466084   111677       30640   50916  ...     2191-5        ug/dL          35.0
13466085   123232       42404   50916  ...     2191-5        ug/dL          16.0
13466086   123232       42404   50916  ...     2191-5        ug/dL          13.0
13466087   122210       65592   50916  ...     2191-5        ug/dL         233.0

[9 rows x 14 columns]

## len_3 단위통일

In [ ]:
len_3=[]
for i in unitdup:
    a = adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()
    if len(a) == 3:
        len_3.append(i)
len_3        

[51464, 51514, 51274, 50804, 50818, 50820, 50821, 50993, 50964, 51000, 50995]

In [ ]:
same_unit = [50804, 50818, 50820, 50821, 50995, 51000, 51274, 51464, 51514, 50993]

In [ ]:
unit_std = []
for u in same_unit:
    unit_std.append(adm_pat_lab_item[adm_pat_lab_item['ITEMID']==u].VALUEUOM.unique()[0])
unit_dict = dict(zip(same_unit, unit_std))
unit_dict    

{50804: 'mEq/L',
 50818: 'mm Hg',
 50820: 'units',
 50821: 'mm Hg',
 50993: 'uU/ML',
 50995: 'ng/dL',
 51000: 'MG/DL',
 51274: 'SECONDS',
 51464: 'EU/dL',
 51514: 'EU/dL'}

In [ ]:
for k, v in unit_dict.items():
    change_unit(k, v)

In [ ]:
# 위 작업 결과 확인
for i in unitdup:
    a = adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique()
    if len(a) != 1: 
        print(i, a)

50964 ['MOSM/L' 'mOsm/kg' 'MOSM/KG']
50889 ['mg/dL' 'mg/L' 'MG/DL' nan]


In [ ]:
# 50964 
adm_pat_lab_item['VALUENUM'] = np.where((adm_pat_lab_item['ITEMID']==50964) & (adm_pat_lab_item['VALUEUOM'] != "MOSM/L"),
                              adm_pat_lab_item['VALUENUM']*0.995,
                              adm_pat_lab_item['VALUENUM'])

In [ ]:
change_unit(50964 , 'MOSM/L')
adm_pat_lab_item[adm_pat_lab_item['ITEMID']==50964]

HADM_ID  SUBJECT_ID  ITEMID  ... LOINC_CODE raw_VALUEUOM  raw_VALUENUM
12998808   104557          30   50964  ...     2692-2       MOSM/L         279.0
12998809   101757          41   50964  ...     2692-2      mOsm/kg         276.0
12998810   116009          62   50964  ...     2692-2      mOsm/kg         284.0
12998811   175016          81   50964  ...     2692-2      mOsm/kg         281.0
12998812   175016          81   50964  ...     2692-2      mOsm/kg         279.0
...           ...         ...     ...  ...        ...          ...           ...
13011598   108710       98669   50964  ...     2692-2      mOsm/kg         292.0
13011599   108710       98669   50964  ...     2692-2      mOsm/kg         287.0
13011600   108710       98669   50964  ...     2692-2      mOsm/kg         290.0
13011601   108710       98669   50964  ...     2692-2      mOsm/kg         290.0
13011602   158912       98749   50964  ...     2692-2      mOsm/kg         375.0

[12795 rows x 14 columns]

In [ ]:
# 50889
adm_pat_lab_item['VALUENUM'] = np.where((adm_pat_lab_item['ITEMID']==50889) & (adm_pat_lab_item['VALUEUOM'] == "mg/dL"),
                              adm_pat_lab_item['VALUENUM']*10,
                              adm_pat_lab_item['VALUENUM'])

In [ ]:
change_unit(50889 , 'mg/L')

In [ ]:
# VALUE=GREATER THAN 300인 데이터 VALUENUM=300으로 입력
adm_pat_lab_item['VALUENUM'] = np.where((adm_pat_lab_item['ITEMID']==50889) 
                                        & ((adm_pat_lab_item['VALUE'] == "GREATER THAN 300")|(adm_pat_lab_item['VALUE'] == "GREATER THAN 30")),
                              300,
                              adm_pat_lab_item['VALUENUM'])

In [ ]:
adm_pat_lab_item[adm_pat_lab_item['ITEMID']==50889].head(50)

HADM_ID  SUBJECT_ID  ITEMID  ... LOINC_CODE raw_VALUEUOM  raw_VALUENUM
13206861   176176          33   50889  ...     1988-5        mg/dL          3.83
13206862   134462         366   50889  ...     1988-5         mg/L        224.10
13206863   116009          62   50889  ...     1988-5        mg/dL          1.14
13206864   164174         129   50889  ...     1988-5         mg/L         74.40
13206865   164174         129   50889  ...     1988-5         mg/L         22.90
13206866   164174         129   50889  ...     1988-5         mg/L         93.20
13206867   107064           6   50889  ...     1988-5        mg/dL          0.20
13206868   182153         454   50889  ...     1988-5        mg/dL          1.31
13206869   166305         457   50889  ...     1988-5        mg/dL          5.83
13206870   166305         457   50889  ...     1988-5        mg/dL         27.71
13206871   157083         491   50889  ...     1988-5        mg/dL          6.12
13206872   182743         164   50889  ...     1988-5        mg/dL         20.00
13206873   153112         171   50889  ...     1988-5        mg/dL         20.61
13206874   105694         223   50889  ...     1988-5         mg/L         45.00
13206875   186251         266   50889  ...     1988-5         mg/L        190.30
13206876   186251         266   50889  ...     1988-5         mg/L        145.70
13206877   106296         269   50889  ...     1988-5        mg/dL           NaN
13206878   173727         271   50889  ...     1988-5        mg/dL           NaN
13206879   170452         588   50889  ...     1988-5         mg/L         19.50
13206880   171266         608   50889  ...     1988-5        mg/dL          0.11
13206881   177056         628   50889  ...     1988-5         mg/L          2.40
13206882   142160         636   50889  ...     1988-5         mg/L        191.60
13206883   124257         644   50889  ...     1988-5         mg/L         12.10
13206884   100262         652   50889  ...     1988-5         mg/L         48.60
13206885   115758         528   50889  ...     1988-5        mg/dL         27.43
13206886   115758         528   50889  ...     1988-5        mg/dL         21.05
13206887   155327         545   50889  ...     1988-5         mg/L         80.50
13206888   104958         558   50889  ...     1988-5         mg/L         14.60
13206889   117029         422   50889  ...     1988-5         mg/L         22.40
13206890   137678         439   50889  ...     1988-5        MG/DL          2.40
13206891   137678         439   50889  ...     1988-5        MG/DL          3.70
13206892   186550         862   50889  ...     1988-5         mg/L         42.00
13206893   187825         784   50889  ...     1988-5        mg/dL          7.74
13206894   100863         797   50889  ...     1988-5         mg/L        218.50
13206895   154890         818   50889  ...     1988-5        mg/dL         26.61
13206896   152970        1205   50889  ...     1988-5         mg/L         48.80
13206897   110021        1206   50889  ...     1988-5        mg/dL          4.38
13206898   190398        1232   50889  ...     1988-5         mg/L         11.80
13206899   192293        1112   50889  ...     1988-5        mg/dL          2.47
13206900   192293        1112   50889  ...     1988-5        mg/dL          9.56
13206901   174792        1068   50889  ...     1988-5        mg/dL           NaN
13206902   174792        1068   50889  ...     1988-5        mg/dL           NaN
13206903   111612         940   50889  ...     1988-5         mg/L         51.30
13206904   185964        1568   50889  ...     1988-5         mg/L        119.60
13206905   133003        1651   50889  ...     1988-5        mg/dL          4.63
13206906   185767        1675   50889  ...     1988-5        mg/dL          6.29
13206907   135253        1694   50889  ...     1988-5        mg/dL          0.06
13206908   111365        1714   50889  ...     1988-5         mg/L        205.30
13206909   118114        1305   50889  ..

In [ ]:
# 마지막 확인 ==> unit 중복 데이터 없음
itemids=adm_pat_lab_item.ITEMID.unique()

a=0
for i in itemids:
    a+=len(adm_pat_lab_item.VALUEUOM[adm_pat_lab_item.ITEMID==i].unique())
a

696

In [ ]:
len(itemids)

696

## adm_pat_lab_item csv로 저장

In [ ]:
adm_pat_lab_item.to_csv("/content/drive/Shareddrives/Multicampus/Preprocessing/unit_all.csv", index=False)

In [ ]:
adm_pat_lab_item

HADM_ID  SUBJECT_ID  ITEMID  ... LOINC_CODE raw_VALUEUOM  raw_VALUENUM
0          165315          22   51071  ...     3349-8          NaN           NaN
1          144383         372   51071  ...     3349-8          NaN           NaN
2          190797          52   51071  ...     3349-8          NaN           NaN
3          172056          64   51071  ...     3349-8          NaN           NaN
4          143430          65   51071  ...     3349-8          NaN           NaN
...           ...         ...     ...  ...        ...          ...           ...
13469464   175201       98573   51169  ...        NaN            %           0.0
13469465   175201       98573   51170  ...        NaN         #/uL          13.0
13469466   175201       98573   51170  ...        NaN         #/uL           2.0
13469467   175201       98573   51170  ...        NaN         #/uL           0.0
13469468   175201       98573   51170  ...        NaN         #/uL           0.0

[13469469 rows x 14 columns]

# (old)단위 통일 작업

In [ ]:
adm_pat_lab_item[adm_pat_lab_item.ITEMID==len_2[0]]
idx=adm_pat_lab_item[adm_pat_lab_item.ITEMID==len_2[0]].index
adm_pat_lab_item.iloc[idx, 6]='NaN'
adm_pat_lab_item.iloc[idx]

HADM_ID  SUBJECT_ID  ITEMID            CHARTTIME VALUE  VALUENUM  \
119775   165315          22   51466  2196-04-09 10:10:00   NEG       NaN   
119776   129635          25   51466  2160-11-02 08:55:00   NEG       NaN   
119777   197661          26   51466  2126-05-10 14:59:00   NEG       NaN   
119778   104557          30   51466  2172-10-15 19:45:00    LG       NaN   
119779   128652          31   51466  2108-08-22 23:35:00   LGE       NaN   
...         ...         ...     ...                  ...   ...       ...   
160113   187987       98790   51466  2200-10-15 03:30:00   NEG       NaN   
160114   105447       98797   51466  2132-12-24 19:15:00    LG       NaN   
160115   101071       98802   51466  2151-03-05 17:45:00    SM       NaN   
160116   122631       98805   51466  2200-09-12 22:11:00   MOD       NaN   
160117   122631       98805   51466  2200-09-14 15:39:00   MOD       NaN   

       VALUEUOM FLAG  LABEL  FLUID    CATEGORY LOINC_CODE  
119775      NaN  NaN  Blood  Urine  Hematology     5794-3  
119776      NaN  NaN  Blood  Urine  Hematology     5794-3  
119777      NaN  NaN  Blood  Urine  Hematology     5794-3  
119778      NaN  NaN  Blood  Urine  Hematology     5794-3  
119779      NaN  NaN  Blood  Urine  Hematology     5794-3  
...         ...  ...    ...    ...         ...        ...  
160113      NaN  NaN  Blood  Urine  Hematology     5794-3  
160114      NaN  NaN  Blood  Urine  Hematology     5794-3  
160115      NaN  NaN  Blood  Urine  Hematology     5794-3  
160116      NaN  NaN  Blood  Urine  Hematology     5794-3  
160117      NaN  NaN  Blood  Urine  Hematology     5794-3  

[40343 rows x 12 columns]

In [ ]:
target=adm_pat_lab_item[adm_pat_lab_item.ITEMID==len_2[1]]
idx=target[target.VALUEUOM.isnull()].index
adm_pat_lab_item.iloc[idx, 6]='#/hpf'
target.VALUEUOM.unique()
target

HADM_ID  SUBJECT_ID  ITEMID            CHARTTIME VALUE  VALUENUM  \
160118   165315          22   51476  2196-04-09 10:10:00    <1       NaN   
160119   197661          26   51476  2126-05-10 14:59:00     2       2.0   
160120   104557          30   51476  2172-10-15 19:45:00     0       0.0   
160121   128652          31   51476  2108-08-22 23:35:00    <1       NaN   
160122   128652          31   51476  2108-08-24 16:30:00     0       0.0   
...         ...         ...     ...                  ...   ...       ...   
195942   187987       98790   51476  2200-10-15 03:30:00   0-2       NaN   
195943   105447       98797   51476  2132-12-24 19:15:00   0-2       NaN   
195944   101071       98802   51476  2151-03-05 17:45:00   0-2       NaN   
195945   122631       98805   51476  2200-09-12 22:11:00     0       0.0   
195946   122631       98805   51476  2200-09-14 15:39:00     0       0.0   

       VALUEUOM FLAG             LABEL  FLUID    CATEGORY LOINC_CODE  
160118    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  
160119    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  
160120    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  
160121    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  
160122    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  
...         ...  ...               ...    ...         ...        ...  
195942    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  
195943    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  
195944    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  
195945    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  
195946    #/hpf  NaN  Epithelial Cells  Urine  Hematology     5787-7  

[35829 rows x 12 columns]

In [ ]:
target=adm_pat_lab_item[adm_pat_lab_item.ITEMID==len_2[2]]
idx=target[target.VALUEUOM.isnull()].index
adm_pat_lab_item.iloc[idx, 6]='mg/dL'
target.VALUEUOM.unique()

array(['mg/dL'], dtype=object)

In [ ]:
target=adm_pat_lab_item[adm_pat_lab_item.ITEMID==len_2[3]]
idx=target[target.VALUEUOM.isnull()].index
adm_pat_lab_item.iloc[idx, 6]='mg/dL'
target.VALUEUOM.unique()

array(['mg/dL', nan], dtype=object)

In [ ]:
target=adm_pat_lab_item[adm_pat_lab_item.ITEMID==len_2[4]]
idx=target[target.VALUEUOM.isnull()].index
adm_pat_lab_item.iloc[idx, 6]='units'
target.VALUEUOM.unique()

array(['units'], dtype=object)

In [ ]:
target=adm_pat_lab_item[adm_pat_lab_item.ITEMID==len_2[5]]
target.VALUEUOM.unique()
target[target.VALUEUOM.isnull()]

idx=target[target.VALUEUOM.isnull()].index
adm_pat_lab_item.iloc[idx, 6]='mg/dL'
target.VALUEUOM.unique()

array(['mg/dL'], dtype=object)

HADM_ID  SUBJECT_ID  ITEMID            CHARTTIME VALUE  VALUENUM  \
407868   165315          22   51492  2196-04-09 10:10:00   NEG       NaN   
407869   129635          25   51492  2160-11-02 08:55:00   NEG       NaN   
407870   197661          26   51492  2126-05-10 14:59:00    TR       NaN   
407871   104557          30   51492  2172-10-15 19:45:00  >300       NaN   
407872   128652          31   51492  2108-08-22 23:35:00   NEG       NaN   
...         ...         ...     ...                  ...   ...       ...   
451903   187987       98790   51492  2200-10-15 03:30:00   NEG       NaN   
451904   105447       98797   51492  2132-12-24 19:15:00    75      75.0   
451905   101071       98802   51492  2151-03-05 17:45:00    25      25.0   
451906   122631       98805   51492  2200-09-12 22:11:00    TR       NaN   
451907   122631       98805   51492  2200-09-14 15:39:00    30      30.0   

       VALUEUOM FLAG    LABEL  FLUID    CATEGORY LOINC_CODE  
407868    mg/dL  NaN  Protein  Urine  Hematology     5804-0  
407869    mg/dL  NaN  Protein  Urine  Hematology     5804-0  
407870    mg/dL  NaN  Protein  Urine  Hematology     5804-0  
407871    mg/dL  NaN  Protein  Urine  Hematology     5804-0  
407872    mg/dL  NaN  Protein  Urine  Hematology     5804-0  
...         ...  ...      ...    ...         ...        ...  
451903    mg/dL  NaN  Protein  Urine  Hematology     5804-0  
451904    mg/dL  NaN  Protein  Urine  Hematology     5804-0  
451905    mg/dL  NaN  Protein  Urine  Hematology     5804-0  
451906    mg/dL  NaN  Protein  Urine  Hematology     5804-0  
451907    mg/dL  NaN  Protein  Urine  Hematology     5804-0  

[44040 rows x 12 columns]